In [2]:
# import pandas as pd

# import boto3
# import re
# import os
# import sagemaker
# from sagemaker import get_execution_role

# bucket = sagemaker.Session().default_bucket()        
# region = boto3.Session().region_name    
# role = get_execution_role()
# bucket

INFO:sagemaker:Created S3 bucket: sagemaker-us-east-1-035504150651


'sagemaker-us-east-1-035504150651'

In [1]:
import numpy as np
from tsfresh import select_features
import xgboost
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib

import math, datetime 

Using matplotlib backend: Qt5Agg


In [2]:
import seaborn as sns
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression

In [3]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
# from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from feature_selection import FeatureSelector

In [5]:
# inputBucket='cmg-kudzu-text-analysis' 

# data_key = 'glue/test/exhaustive_extracted_features.pkl'
# exhaustive_extracted_features_s3file = '{}/{}'.format(inputBucket, data_key)

# data_key = 'glue/test/y_df_for_feature_selection.pkl'
# y_df_for_feature_selection_s3file = '{}/{}'.format(inputBucket, data_key)

# data_key = 'glue/test/full_features.csv'
# original_full_features_s3file = '{}/{}'.format(inputBucket, data_key)

# print('Location {} {}'.format(exhaustive_extracted_features_s3file,y_df_for_feature_selection_s3file))

Location cmg-kudzu-text-analysis/glue/test/exhaustive_extracted_features.pkl cmg-kudzu-text-analysis/glue/test/y_df_for_feature_selection.pkl


In [6]:
# import s3fs
# import pandas as pd
# fs = s3fs.S3FileSystem(anon=False)

# # Pickle
# with fs.open(exhaustive_extracted_features_s3file) as f:
#     full_features = pd.read_pickle(f)

# with fs.open(y_df_for_feature_selection_s3file) as f:
#     y_df = pd.read_pickle(f)

# # with fs.open(original_full_features_s3file) as f:
# #     original_full_features = pd.read_csv(f)


In [4]:
# all extracted features from tsfresh
# the extracted features are the same for all models
full_features = pd.read_pickle('/Users/Rohil/Documents/iGEM/yemen/exhaustive_extracted_features_new.pkl')

# test values for different time frames (in a nice format)
y_df = pd.read_pickle('/Users/Rohil/Documents/iGEM/yemen/y_df_for_feature_selection_new.pkl')

In [5]:
full_features = pd.concat([full_features, pd.get_dummies(full_features.gov_iso)], axis=1)

In [6]:
# ignoring anything that happens before June 19, as there's not enough data for feature calculation
full_features = full_features[full_features.date>='2017-07-01']
y_df = y_df[y_df.date>='2017-07-01']

In [7]:
ESTIMATORS = 1000
EARLY_STOPPING_ROUNDS = 50

In [8]:
def scale_features(scaler, X):

    continuous_cols = [col for col in X.columns if 'YE-' not in col]
    
    X_scaled = X
    
    X_scaled[continuous_cols] = scaler.transform(X[continuous_cols])
    
    return (X_scaled)

In [9]:
def train_cv_test_split(data, cv_split_date='2017-09-17', test_split_date='2017-11-06'):
    
    data_train, data_cv, data_test = data[(data.date<cv_split_date)], data[(data.date<test_split_date) & (data.date>=cv_split_date)], data[ (data.date>=test_split_date) ]
    
    return (data_train.drop('date', axis=1), data_cv.drop('date', axis=1), data_test.drop('date', axis=1))

In [10]:
HOLD_OUT_WINDOW = 100
ROLLING_WINDOW_SIZE = 60
WINDOW = round(ROLLING_WINDOW_SIZE/3)

def getMaxDate():
    return full_features.date.max() - datetime.timedelta(days=HOLD_OUT_WINDOW)

def getMinDate():
    return full_features.date.min()

delta = getMaxDate() - getMinDate()
print('Start {} Stop {} No of days {}'.format(getMinDate(), getMaxDate(), delta.days))

number_rolling_windows = math.ceil(round(delta.days/WINDOW))  - 1
number_rolling_windows


Start 2017-07-01 00:00:00 Stop 2017-11-10 00:00:00 No of days 132


6

In [11]:
def getHoldOutDate():
    start = getMaxDate() + datetime.timedelta(days=1)
    end   = full_features.date.max() 
    
    return (start, end)

In [12]:
def getRollingWindowDates(idx):
    maxDate = getMaxDate()  
    minDate = getMinDate() 
    
    trainStart = minDate
    trainStop  = minDate + datetime.timedelta(days=WINDOW*idx)
    
    validationStart  = trainStop + datetime.timedelta(days=1)
    validationStop   = validationStart + datetime.timedelta(days=WINDOW)
    
    testStart  = validationStop + datetime.timedelta(days=1)
    testStop   = testStart + datetime.timedelta(days=WINDOW)
    
    if (maxDate - testStop).days < WINDOW:
        print('Rolling window to end date')
        testStop = maxDate
        
    print('Train [{} {}] Val [{} {}] Test [{} {}]'.format(trainStart.date(), trainStop.date(), 
                                                       validationStart.date(), validationStop.date(), 
                                                       testStart.date(), testStop.date()))
    
    return (trainStart, trainStop, validationStart, validationStop, testStart, testStop)

In [13]:
def getRollingWindow(data, trainStart, trainStop, validationStart, validationStop, testStart, testStop):
    
    train = data[(data.date >= trainStart) & (data.date <= trainStop)]
    val   = data[(data.date >= validationStart) & (data.date <= validationStop)]
    test  = data[(data.date >= testStart) & (data.date <= testStop)] 
    
    print('Window Train/Val/Test shape {} {} {}'.format(train.shape, val.shape, test.shape))
    
    return (train.drop('date', axis=1), val.drop('date', axis=1), test.drop('date', axis=1))

In [14]:
def getHoldOutData(data):
    minDate = getMinDate() 
    start, end = getHoldOutDate()

    train = data[(data.date >= minDate) & (data.date < start)]
    test =  data[(data.date >= start) & (data.date <= end)]
    
    return (train.drop('date', axis=1), test.drop('date', axis=1))

In [15]:
number_rolling_windows, round((ROLLING_WINDOW_SIZE)/3), getHoldOutDate()

(6, 20, (Timestamp('2017-11-11 00:00:00'), Timestamp('2018-02-18 00:00:00')))

In [16]:
class TsFresh():
    def __init__(self):
        pass
    
    def postProcessor(self, X, y, dateSeries):
        self.selected_features = select_features(X, y, fdr_level=0.001)
        print('Selected features {}'.format(self.selected_features.shape))      
        self.selected_features = pd.concat([dateSeries, self.selected_features], axis=1)

In [17]:
class OptimizeFeatures():
    def __init__(self):
        pass
        
    def selectFeatures(self, X, y):
        self.X  = X
        self.y  = y
        self.fs = FeatureSelector(data = X, labels = y)        
    
    def identifyCollinearFeatures(self, correlation_threshold=0.975):
        self.fs.identify_collinear(correlation_threshold)
    
    def collinerFeaturesColumnsToKeep(self):
        return self.fs.ops['collinear']
    
    def removeCollinerFeatures(self):
        self.cols_to_keep = set(self.X.columns) - set(self.fs.ops['collinear'])
        self.corr_selected_features = self.X[list(self.cols_to_keep)]


In [36]:
class ModelCustomRegressor():
    
    def __init__(self):
        pass
    
    def extract(self, model_params, X_train, X_cv, X_test, y_train, y_cv, y_test):
        print('Creating baseline model to extract features')
        
        X_train_cv = X_train.append(X_cv)
        y_train_cv = y_train.append(y_cv)
        
        scaler = StandardScaler()
        continuous_cols = [col for col in X_train_cv.columns if 'YE-' not in col]
        scaler.fit(X_train_cv[continuous_cols])

        X_train_cv, X_test = scale_features(scaler, X_train_cv), scale_features(scaler, X_test)
        
        print('all features {}'.format(X_train_cv.shape))
            
        eval_set = [(X_test, y_test)]
    
        self.feature_importance_df = pd.DataFrame(index = X_train.columns)
        
        if model_params == None:
            
            model_params = { 'n_estimators' :  ESTIMATORS,
                              'alpha' : 0.816145500886388,
                              'eta' : 0.12130551203237118,
                              'gamma' : 4,
                              'rate_drop' : 0.3,
                              'learning_rate': 0.01, 
                              'max_depth': 9, 
                              'min_child_weight': 7.480659372857079,
                              'subsample': 0.34043298101792013,
                              'colsample_bytree': 0.8995724457113121,
                              'tweedie_variance_power' : 1.4,
                              'n_jobs' : -1,
                              'verbose' : False,
                              'eval_metric':'mae', 
                              'eval_set':eval_set, 
                              'early_stopping_rounds':EARLY_STOPPING_ROUNDS}
        
        self.regressor = xgboost.XGBRegressor(**model_params)                                      
        
        self.regressor.fit(X_train_cv, y_train_cv, eval_metric='mae', 
                                  eval_set=eval_set, 
                                  early_stopping_rounds=EARLY_STOPPING_ROUNDS, verbose=False)

        preds = self.regressor.predict((X_test))
        
        self.feature_importance_df['threshold'] = self.regressor.feature_importances_
        
        self.thresholds = np.unique(self.regressor.feature_importances_)
        self.thresholds.sort();
        self.thresholds = self.thresholds[::-1][:30]
        
        print('Thresholds are {} {} {}'.format(len(self.regressor.feature_importances_), len(self.thresholds), self.thresholds))

In [19]:
class RollingWindowCrossValidation():
    
    def __init__(self, full_features, corr_selected_features_bi_week, corr_selected_features_bi_week_y):
        self.full_features = full_features
        self.corr_selected_features_bi_week = corr_selected_features_bi_week
        self.corr_selected_features_bi_week_y   = corr_selected_features_bi_week_y
        pass
    
    def extract(self, regressor, thresh):
        
        mae_list = []
        ev_list = []
        
        for idx in range(3, number_rolling_windows):
            print('CV - Window {}'.format(idx))
            trainStart, trainStop, validationStart, validationStop, testStart, testStop = getRollingWindowDates(idx)
            
            X_train, X_cv, X_test = getRollingWindow(self.corr_selected_features_bi_week, 
                                                     trainStart, trainStop, 
                                                     validationStart, validationStop, testStart, testStop)
            y_train, y_cv, y_test = getRollingWindow(self.corr_selected_features_bi_week_y, 
                                                     trainStart, trainStop, 
                                                     validationStart, validationStop, testStart, testStop)
            
            X_train_cv = X_train.append(X_cv)
            y_train_cv = y_train.append(y_cv)
            
            scaler = StandardScaler()
            continuous_cols = [col for col in X_train_cv.columns if 'YE-' not in col]
            scaler.fit(X_train_cv[continuous_cols])

            X_train_cv, X_test = scale_features(scaler, X_train_cv), scale_features(scaler, X_test)
        
            print('X_train_cv  {}'.format(X_train_cv.shape))
            
            # select features using threshold
            selection = SelectFromModel(regressor, threshold=thresh, prefit=True)
            select_X_train_cv = selection.transform(X_train_cv)
            
            select_X_test = selection.transform(X_test)
            
            eval_set = [(select_X_test, y_test)]
            
            # train model
            selection_model = xgboost.XGBRegressor(n_estimators = ESTIMATORS,
                                      alpha = 0.816145500886388,
                                      eta = 0.12130551203237118,
                                      gamma = 4,
                                      rate_drop = 0.3,
                                      learning_rate= 0.01, 
                                      max_depth= 9, 
                                      min_child_weight= 7.480659372857079,
                                      subsample= 0.34043298101792013,
                                      colsample_bytree= 0.8995724457113121,
                                      tweedie_variance_power = 1.4,
                                      n_jobs = -1,
                                      verbose = False,
                                      eval_metric='mae', 
                                      eval_set=eval_set, 
                                      early_stopping_rounds=EARLY_STOPPING_ROUNDS)
            
            selection_model.fit(select_X_train_cv, y_train_cv, eval_metric='mae', 
                                      eval_set=eval_set, 
                                      early_stopping_rounds=50,
                                      verbose=False)
            # eval model
            y_pred = selection_model.predict(select_X_test)
           
            mae = mean_absolute_error(y_test, y_pred)
            ev = explained_variance_score(y_test, y_pred)
            print("%d Thresh=%.5f, n=%d, mae: %.3f, mae: %.4f" % (idx, thresh, select_X_train_cv.shape[1], mae, ev))
            
            mae_list.append(mae)
            ev_list.append(ev)
        
        lst = [thresh, select_X_train_cv.shape[1]] + mae_list + ev_list
        return_dict = dict(zip(['threshold', 'num_features', 'mae3', 'mae4', 'mae5', 'ev3', 'ev4', 'ev5'], lst))
        print (return_dict)
        return (return_dict)
        

In [20]:
class RollingCustomFeatureExtractor():
    
    def __init__(self, full_features, corr_selected_features_bi_week, corr_selected_features_bi_week_y):
        self.full_features = full_features
        self.corr_selected_features_bi_week  = corr_selected_features_bi_week
        self.corr_selected_features_bi_week_y   = corr_selected_features_bi_week_y
        pass
    
    def extract(self, model_params):
        X = self.corr_selected_features_bi_week 
        y = self.corr_selected_features_bi_week_y 
        
        X_train, X_cv, X_test = train_cv_test_split(X)
        y_train, y_cv, y_test = train_cv_test_split(y)
        
        self.mcr = ModelCustomRegressor()
        self.mcr.extract(model_params, X_train, X_cv, X_test, y_train, y_cv, y_test) 
            
        thresholds = self.mcr.thresholds        
    
        self.rwcv = RollingWindowCrossValidation(self.full_features, 
                                                 self.corr_selected_features_bi_week, 
                                                 self.corr_selected_features_bi_week_y)
        
        self.summary = pd.DataFrame(columns = ['threshold', 'num_features', 'mae3', 'mae4', 'mae5', 'ev3', 'ev4', 'ev5' ])
        
        for thresh in thresholds:
            return_dict = self.rwcv.extract(self.mcr.regressor, thresh)
            
            self.summary = self.summary.append(return_dict, ignore_index = True)
            
            print("\n")
            
        print(self.summary.head(20))


In [21]:
def set_style(color):
    plt.style.use(['seaborn-' + color, 'seaborn-paper'])

In [22]:
def plot_pred_against_actual(pred_crosstab_dict, true_crosstab, y_label):
    
    set_style('white')
    
    fig, ax = plt.subplots(21,1,figsize = (6,15), sharex=True)
    
    cols = true_crosstab.columns
    
    minDate = getMinDate() 
    start, end = getHoldOutDate()
    
    train_pred_crosstab = {}
    test_pred_crosstab = {}
    
    for key, value in pred_crosstab_dict.items():

        train_pred_crosstab[key] = pred_crosstab_dict[key].loc[(pred_crosstab_dict[key].index >= minDate) & (pred_crosstab_dict[key].index < start)]
        test_pred_crosstab[key] =  pred_crosstab_dict[key].loc[(pred_crosstab_dict[key].index >= start) & (pred_crosstab_dict[key].index <= end)]
    
    for i in range(0,21):
        
        true_crosstab[cols[i]].plot(kind='line', ax = ax[i], label = 'true_val', legend = True, color = 'red')

        ax[i].set_prop_cycle('color', ['seagreen', 'blue', 'plum', 'magenta'])
        
        for key, value in pred_crosstab_dict.items():
                    
            train_pred_crosstab[key][cols[i]].plot(kind='line', ax = ax[i], label= key + ' train-pred',linestyle= '-.', legend = True)
        
            test_pred_crosstab[key][cols[i]].plot(kind='line', ax = ax[i], label= key + ' test-pred', linestyle= '-.',legend = True)   

          
        ax[i].legend().set_visible(False)
        ax[i].set_ylabel(cols[i])
        ax[i].yaxis.set_label_position('right')
        ax[i].spines['right'].set_visible(False)
        ax[i].spines['top'].set_visible(False)
        ax[i].spines['bottom'].set_visible(True)
        
    ax[10].legend().set_visible(True)
    ax[10].legend(fontsize=10, loc='center left', bbox_to_anchor=(1.05, 0.5))
 
    fig.subplots_adjust(hspace = .2)    
        
    fig.savefig('/Users/Rohil/Documents/iGEM/yemen/' + y_label + '_deployed.png', dpi = 500, bbox_inches = 'tight')
    
    plt.close('all')

In [35]:
class DeployRegressor():
    
    def __init__(self):
        pass
    
    def execute(self, model_params, X_train, X_test, y_train, y_test, y_to_plot, y_bi_week_label):
        print('Creating baseline model to train on final features')
        
        continuous_cols = [col for col in X_train.columns if 'YE-' not in col]

        scaler = StandardScaler()
        scaler.fit(X_train[continuous_cols])

        X_train, X_test = scale_features(scaler, X_train), scale_features(scaler, X_test)
                    
        if model_params == None:
            model_params =  {'n_estimators' : 150,
                                  'alpha' : 0.39570169370620345, #0.816145500886388,
                                  'eta' : 0.1024118543732113, #0.12130551203237118,
                                  'gamma' : 4,
                                  'rate_drop' : 0.3,
                                  'learning_rate' : 0.01, 
                                  'max_depth' : 8, #9, 
                                  'min_child_weight' : 3.666935706656843, #7.480659372857079,
                                  'subsample' : 0.3075355396312034, #0.34043298101792013,
                                  'colsample_bytree' : 0.4093269448033341, #0.8995724457113121,
                                  'tweedie_variance_power' : 1.4,
                                  'n_jobs' : -1,
                                  'verbose' : False,
                                  'eval_metric' :'mae'}   
        
        self.xgb = xgboost.XGBRegressor(**model_params) 
        
        self.linreg = LinearRegression()
        
        self.xgb.fit(X_train, y_train, eval_metric='mae', verbose=False)
        self.linreg.fit(X_train,y_train)

        y_pred_xgb = self.xgb.predict((X_test))
        y_pred_all_xgb = self.xgb.predict(X_train.append(X_test))
        
        y_pred_linreg = self.linreg.predict((X_test))
        y_pred_all_linreg = self.linreg.predict(X_train.append(X_test))
        
        y_test_merged = y_test.merge(y_to_plot, how='left', left_index=True, right_index=True, copy=False)
        
        y_pred_all_xgb = pd.DataFrame(y_pred_all_xgb, columns=[y_bi_week_label], index = y_to_plot.index)
        y_pred_all_xgb[['gov_iso', 'date']] = y_to_plot[['gov_iso', 'date']]
        
        y_pred_all_linreg = pd.DataFrame(y_pred_all_linreg, columns=[y_bi_week_label], index = y_to_plot.index)
        y_pred_all_linreg[['gov_iso', 'date']] = y_to_plot[['gov_iso', 'date']]
        
        fig, ax = plt.subplots(figsize=(6,15))
        
        true_val_pivot = y_to_plot.pivot_table(index = 'date', columns = 'gov_iso', values = y_bi_week_label, aggfunc='sum')
        xgb_pred_val_pivot = y_pred_all_xgb.pivot_table(index = 'date', columns = 'gov_iso', values = y_bi_week_label, aggfunc='sum')
        linreg_pred_val_pivot = y_pred_all_linreg.pivot_table(index = 'date', columns = 'gov_iso', values = y_bi_week_label, aggfunc='sum')
 
        pred_crosstab_dict = {'xgboost':xgb_pred_val_pivot}#, 'linreg':linreg_pred_val_pivot}

        plot_pred_against_actual(pred_crosstab_dict, true_val_pivot, y_bi_week_label)
        
        fig1, ax1 = plt.subplots()
        xgboost.plot_importance(self.xgb, ax=ax1)
        fig1.savefig('/Users/Rohil/Documents/iGEM/yemen/' + y_bi_week_label + '_deployed_feature_importances.png', dpi = 300, bbox_inches = 'tight')
        plt.close('all')
        
        mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
        ev_xgb = explained_variance_score(y_test, y_pred_xgb)
        
        mae_linreg = mean_absolute_error(y_test, y_pred_linreg)
        ev_linreg = explained_variance_score(y_test, y_pred_linreg)

        print('y-test mean {}, y-test std {}'.format(np.mean(y_test.values), np.std(y_test.values)))
        print('xgb mae {}, xgb ev {}'.format(mae_xgb, ev_xgb))
        print('linreg mae {}, linreg ev {}'.format(mae_linreg, ev_linreg))

        

In [34]:
class Orchestrator():
    
    def __init__(self, full_features, full_data_bi_week, y_bi_week_label):
        self.full_features = full_features
        self.full_data_bi_week = full_data_bi_week
        self.y_bi_week_label   = y_bi_week_label
        pass
    
    def runTsFresh(self):
        print('Running TSFresh....')
        
        X_ts = self.full_data_bi_week[self.full_features.columns].drop(columns=['date', 'gov_iso', 'days_from'])
        y_ts = self.full_data_bi_week[self.y_bi_week_label]
        dateSeries_ts = self.full_data_bi_week.date
        
        self.tf = TsFresh()
        self.tf.postProcessor(X_ts, y_ts, dateSeries_ts)
        
        print('Finished running TSFresh....')
        
    def runOptimizeFeatures(self):
        print('Running Feature Selection module ....')
        
        y = self.full_data_bi_week[self.y_bi_week_label]
            
        self.op = OptimizeFeatures()
        self.op.selectFeatures(self.tf.selected_features, y)
        self.op.identifyCollinearFeatures(0.975)
        
        self.op.removeCollinerFeatures()
        print('Original {} and after {}'.format(self.op.X.shape, self.op.corr_selected_features.shape))
        
        print('Finished running Feature Selection ....')
        
    def runRollingCustomFeatureExtractor(self, model_params):
        print('Running custom feature selection module ....')
            
        self.corr_selected_features_bi_week = self.op.corr_selected_features
        self.corr_selected_features_bi_week_y = self.full_data_bi_week[['date', self.y_bi_week_label]]

        #corr_selected_features_4_6 = pd.concat([full_data_4_6.date, corr_selected_features_4_6], axis=1)
        #corr_selected_features_4_6.date.head()

        """
        X = corr_selected_features_4_6 
        y = corr_selected_features_4_6_y 
        
        X_train, X_cv, X_test = train_cv_test_split(X)
        y_train, y_cv, y_test = train_cv_test_split(y)
        """

        self.rcfe = RollingCustomFeatureExtractor(self.full_features, 
                                                  self.corr_selected_features_bi_week, 
                                                  self.corr_selected_features_bi_week_y)
        self.rcfe.extract(model_params)

    def deploy(self, threshold, model_params):
        
        #selected_features_from_threshold = list(self.rcfe.mcr.feature_importance_df[self.rcfe.mcr.feature_importance_df.threshold >= 0.003145].index)
        selected_features_from_threshold = list(self.rcfe.mcr.feature_importance_df[self.rcfe.mcr.feature_importance_df.threshold >= threshold].index)
        selected_features_from_threshold.append('date')

        X = self.corr_selected_features_bi_week[selected_features_from_threshold]
        y = self.corr_selected_features_bi_week_y 
        
        X['days_from'] = self.full_data_bi_week.days_from
        categorical_cols =  [col for col in self.full_data_bi_week.columns if 'YE-' in col]
        
        X_train, X_hold_test = getHoldOutData(X)
        y_train, y_hold_test = getHoldOutData(y)

        y_to_plot = self.corr_selected_features_bi_week_y.merge(self.full_features, on = 'date', left_index=True, right_index=True, how = 'left')[[self.y_bi_week_label, 'gov_iso', 'date']]
                
        self.dr = DeployRegressor()
        self.dr.execute(model_params, X_train, X_hold_test, y_train, y_hold_test, y_to_plot, self.y_bi_week_label)
        
        

In [26]:
print (full_features.date.min())
print (full_features.date.max())

2017-07-01 00:00:00
2018-02-18 00:00:00


In [27]:
y1_2 = y_df[['date', 'gov_iso', 'week_1_to_2_cases']]
y2_4 = y_df[['date', 'gov_iso', 'week_2_to_4_cases']]
y4_6 = y_df[['date', 'gov_iso', 'week_4_to_6_cases']]
y6_8 = y_df[['date', 'gov_iso', 'week_6_to_8_cases']]

In [28]:
full_data_1_2 = y1_2.dropna().merge(full_features, how = 'left', on = ['gov_iso', 'date']).sort_values('date')
full_data_2_4 = y2_4.dropna().merge(full_features, how = 'left', on = ['gov_iso', 'date']).sort_values('date')
full_data_4_6 = y4_6.dropna().merge(full_features, how = 'left', on = ['gov_iso', 'date']).sort_values('date')
full_data_6_8 = y6_8.dropna().merge(full_features, how = 'left', on = ['gov_iso', 'date']).sort_values('date')

In [37]:
# running feature selection for 1-2 week model

In [38]:
orchestrator12 = Orchestrator(full_features, full_data_1_2, 'week_1_to_2_cases' )

In [39]:
orchestrator12.runTsFresh()

Running TSFresh....
Selected features (4599, 15252)
Finished running TSFresh....


In [40]:
orchestrator12.runOptimizeFeatures()

Running Feature Selection module ....
4825 features with a correlation magnitude greater than 0.97.

Original (4599, 15253) and after (4599, 10428)
Finished running Feature Selection ....


In [62]:
X_12_preselect = orchestrator12.op.corr_selected_features
y_12_preselect = orchestrator12.full_data_bi_week[['date', orchestrator12.y_bi_week_label]]

In [63]:
X_12_preselect.to_csv('/Users/Rohil/Documents/iGEM/yemen/sagemaker_input/X_12_preselect.csv')
y_12_preselect.to_csv('/Users/Rohil/Documents/iGEM/yemen/sagemaker_input/y_12_preselect.csv')

In [ ]:
orchestrator12.runRollingCustomFeatureExtractor()

In [ ]:
orchestrator12.rcfe.summary[['mean_mae', 'mean_ev'] ]= pd.concat([orchestrator12.rcfe.summary[['mae3', 'mae4', 'mae5']].mean(axis=1), orchestrator12.rcfe.summary[['ev3', 'ev4', 'ev5']].mean(axis=1)], axis=1)

In [ ]:
orchestrator12.rcfe.summary

In [41]:
# running feature selection for 2-4 week model

In [42]:
orchestrator24 = Orchestrator(full_features, full_data_2_4, 'week_2_to_4_cases' )

In [43]:
orchestrator24.runTsFresh()

Running TSFresh....
Selected features (4305, 15146)
Finished running TSFresh....


In [44]:
orchestrator24.runOptimizeFeatures()

Running Feature Selection module ....
4810 features with a correlation magnitude greater than 0.97.

Original (4305, 15147) and after (4305, 10337)
Finished running Feature Selection ....


In [64]:
X_24_preselect = orchestrator24.op.corr_selected_features
y_24_preselect = orchestrator24.full_data_bi_week[['date', orchestrator24.y_bi_week_label]]

In [65]:
X_24_preselect.to_csv('/Users/Rohil/Documents/iGEM/yemen/sagemaker_input/X_24_preselect.csv')
y_24_preselect.to_csv('/Users/Rohil/Documents/iGEM/yemen/sagemaker_input/y_24_preselect.csv')

In [243]:
orchestrator24.runRollingCustomFeatureExtractor()

Running custom feature selection module ....
Creating baseline model to extract features
all features (2940, 7904)
Thresholds are 7904 30 [0.01773208 0.01341082 0.01206974 0.00864253 0.0065564  0.00640739
 0.00536433 0.00476829 0.00432126 0.00417225 0.00387424 0.00372523
 0.00357622 0.00342721 0.0032782  0.00312919 0.00283117 0.00268216
 0.00253315 0.00238415 0.00223514 0.00208613 0.00193712 0.00178811
 0.0016391  0.00149009 0.00134108 0.00119207 0.00104306 0.00089405]
CV - Window 3
Train [2017-06-19 2017-08-18] Val [2017-08-19 2017-09-08] Test [2017-09-09 2017-09-29]
Window Train/Val/Test shape (1281, 7905) (441, 7905) (441, 7905)
Window Train/Val/Test shape (1281, 2) (441, 2) (441, 2)
X_train_cv  (1722, 7904)
3 Thresh=0.01773, n=1, mae: 6.748, mae: 0.6159
CV - Window 4
Train [2017-06-19 2017-09-07] Val [2017-09-08 2017-09-28] Test [2017-09-29 2017-10-19]
Window Train/Val/Test shape (1701, 7905) (441, 7905) (441, 7905)
Window Train/Val/Test shape (1701, 2) (441, 2) (441, 2)
X_train_cv

Window Train/Val/Test shape (1281, 7905) (441, 7905) (441, 7905)
Window Train/Val/Test shape (1281, 2) (441, 2) (441, 2)
X_train_cv  (1722, 7904)
3 Thresh=0.00328, n=19, mae: 4.901, mae: 0.7864
CV - Window 4
Train [2017-06-19 2017-09-07] Val [2017-09-08 2017-09-28] Test [2017-09-29 2017-10-19]
Window Train/Val/Test shape (1701, 7905) (441, 7905) (441, 7905)
Window Train/Val/Test shape (1701, 2) (441, 2) (441, 2)
X_train_cv  (2142, 7904)
4 Thresh=0.00328, n=19, mae: 4.102, mae: 0.7836
CV - Window 5
Rolling window to end date
Train [2017-06-19 2017-09-27] Val [2017-09-28 2017-10-18] Test [2017-10-19 2017-11-10]
Window Train/Val/Test shape (2121, 7905) (441, 7905) (483, 7905)
Window Train/Val/Test shape (2121, 2) (441, 2) (483, 2)
X_train_cv  (2562, 7904)
5 Thresh=0.00328, n=19, mae: 2.944, mae: 0.8524
{'threshold': 0.0032782, 'num_features': 19, 'mae3': 4.901316200112907, 'mae4': 4.102468090034351, 'mae5': 2.9439071223157915, 'ev3': 0.7863642537693181, 'ev4': 0.7835707336466499, 'ev5': 0

Train [2017-06-19 2017-09-27] Val [2017-09-28 2017-10-18] Test [2017-10-19 2017-11-10]
Window Train/Val/Test shape (2121, 7905) (441, 7905) (483, 7905)
Window Train/Val/Test shape (2121, 2) (441, 2) (483, 2)
X_train_cv  (2562, 7904)
5 Thresh=0.00104, n=164, mae: 2.925, mae: 0.8476
{'threshold': 0.0010430636, 'num_features': 164, 'mae3': 4.876025152968054, 'mae4': 4.320593995628888, 'mae5': 2.9251612077266276, 'ev3': 0.7897898265252128, 'ev4': 0.7641250058696215, 'ev5': 0.8475513362781899}


CV - Window 3
Train [2017-06-19 2017-08-18] Val [2017-08-19 2017-09-08] Test [2017-09-09 2017-09-29]
Window Train/Val/Test shape (1281, 7905) (441, 7905) (441, 7905)
Window Train/Val/Test shape (1281, 2) (441, 2) (441, 2)
X_train_cv  (1722, 7904)
3 Thresh=0.00089, n=216, mae: 4.915, mae: 0.7893
CV - Window 4
Train [2017-06-19 2017-09-07] Val [2017-09-08 2017-09-28] Test [2017-09-29 2017-10-19]
Window Train/Val/Test shape (1701, 7905) (441, 7905) (441, 7905)
Window Train/Val/Test shape (1701, 2) (441

In [244]:
orchestrator24.rcfe.summary[['mean_mae', 'mean_ev'] ]= pd.concat([orchestrator24.rcfe.summary[['mae3', 'mae4', 'mae5']].mean(axis=1), orchestrator24.rcfe.summary[['ev3', 'ev4', 'ev5']].mean(axis=1)], axis=1)

In [245]:
orchestrator24.rcfe.summary

threshold  num_features      mae3      mae4      mae5       ev3       ev4  \
0    0.017732           1.0  6.747872  5.396475  3.438221  0.615922  0.633925   
1    0.013411           2.0  6.234977  5.581894  3.323022  0.633072  0.630171   
2    0.012070           3.0  6.092285  5.586668  3.386633  0.643673  0.632200   
3    0.008643           4.0  6.259597  5.751100  3.412988  0.632744  0.606861   
4    0.006556           5.0  6.042066  5.970795  3.433921  0.653784  0.579620   
5    0.006407           6.0  6.233433  5.905804  3.515189  0.642599  0.589103   
6    0.005364           7.0  5.923343  4.958234  3.162893  0.663140  0.671922   
7    0.004768           8.0  5.092201  4.971790  2.885167  0.750168  0.637935   
8    0.004321           9.0  5.216000  4.825458  2.721812  0.750507  0.655374   
9    0.004172          11.0  5.114897  4.278828  2.456696  0.760625  0.732222   
10   0.003874          15.0  5.105910  4.038561  2.523220  0.757218  0.763028   
11   0.003725          16.0  5.084973  4.206862  2.447954  0.758678  0.739900   
12   0.003576          17.0  4.990175  4.178479  2.555061  0.781650  0.759158   
13   0.003427          18.0  5.024724  4.273525  2.585658  0.773216  0.753982   
14   0.003278          19.0  4.901316  4.102468  2.943907  0.786364  0.783571   
15   0.003129          24.0  4.612334  4.098055  2.945218  0.792963  0.789691   
16   0.002831          28.0  4.783841  4.038668  2.929671  0.801903  0.798701   
17   0.002682          31.0  4.895711  4.042405  3.012085  0.792052  0.795911   
18   0.002533          37.0  4.825850  4.026894  2.966053  0.794714  0.795889   
19   0.002384          43.0  4.686280  3.883897  3.043211  0.809042  0.818277   
20   0.002235          49.0  4.978181  3.916959  2.944349  0.768919  0.813911   
21   0.002086          50.0  5.141728  4.031235  2.863621  0.750977  0.803255   
22   0.001937          54.0  4.939105  3.938919  2.799565  0.767119  0.813391   
23   0.001788          65.0  4.959423  3.968668  2.730477  0.775961  0.817479   
24   0.001639          75.0  4.953548  3.946413  2.806701  0.784356  0.818522   
25   0.001490          89.0  4.934955  4.059369  2.841104  0.782246  0.803005   
26   0.001341         100.0  4.747066  4.091466  2.846932  0.791926  0.794916   
27   0.001192         124.0  4.808882  4.252809  2.866095  0.782394  0.785954   
28   0.001043         164.0  4.876025  4.320594  2.925161  0.789790  0.764125   
29   0.000894         216.0  4.915364  4.155280  2.895617  0.789290  0.785396   

         ev5  mean_mae   mean_ev  
0   0.763407  5.194190  0.671085  
1   0.761308  5.046631  0.674850  
2   0.748123  5.021862  0.674666  
3   0.742324  5.141228  0.660643  
4   0.742820  5.148928  0.658742  
5   0.738998  5.218142  0.656900  
6   0.824530  4.681490  0.719864  
7   0.853001  4.316386  0.747035  
8   0.864715  4.254423  0.756865  
9   0.884682  3.950141  0.792510  
10  0.883486  3.889230  0.801244  
11  0.892646  3.913263  0.797075  
12  0.888401  3.907905  0.809736  
13  0.882097  3.961302  0.803098  
14  0.852358  3.982564  0.807431  
15  0.834684  3.885202  0.805779  
16  0.828425  3.917393  0.809676  
17  0.821981  3.983400  0.803315  
18  0.839733  3.939599  0.810112  
19  0.840631  3.871129  0.822650  
20  0.856566  3.946497  0.813132  
21  0.865112  4.012195  0.806448  
22  0.870988  3.892530  0.817166  
23  0.881429  3.886189  0.824956  
24  0.860057  3.902221  0.820978  
25  0.867906  3.945143  0.817719  
26  0.866148  3.895155  0.817663  
27  0.853831  3.975929  0.807393  
28  0.847551  4.040593  0.800489  
29  0.849564  3.988754  0.808084

In [294]:
selected_features24 = ['date'] + list(orchestrator24.rcfe.mcr.feature_importance_df[orchestrator24.rcfe.mcr.feature_importance_df.threshold >= 0.002384].index)

In [295]:
orchestrator24.corr_selected_features_bi_week[selected_features24].to_csv('/Users/Rohil/Documents/iGEM/yemen/sagemaker_input/X_2_4.csv')

In [296]:
orchestrator24.corr_selected_features_bi_week_y.to_csv('/Users/Rohil/Documents/iGEM/yemen/sagemaker_input/y_2_4.csv')

In [297]:
orchestrator24.deploy(0.002384)

C:\Users\Rohil\Anaconda3\lib\site-packages\ipykernel_launcher.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Creating baseline model to train on final features
y-test mean 5.756598093022273, y-test std 6.644771694378367
xgb mae 2.6350936469411717, xgb ev 0.6847313632395204
linreg mae 7.18744640746357, linreg ev -0.43011188106644305


In [45]:
# running feature selection for 4-6 week model

In [46]:
orchestrator46 = Orchestrator(full_features, full_data_4_6, 'week_4_to_6_cases' )

In [47]:
orchestrator46.runTsFresh()

Running TSFresh....
Selected features (4011, 15156)
Finished running TSFresh....


In [48]:
orchestrator46.runOptimizeFeatures()

Running Feature Selection module ....
4773 features with a correlation magnitude greater than 0.97.

Original (4011, 15157) and after (4011, 10384)
Finished running Feature Selection ....


In [66]:
X_46_preselect = orchestrator46.op.corr_selected_features
y_46_preselect = orchestrator46.full_data_bi_week[['date', orchestrator46.y_bi_week_label]]

In [67]:
X_46_preselect.to_csv('/Users/Rohil/Documents/iGEM/yemen/sagemaker_input/X_46_preselect.csv')
y_46_preselect.to_csv('/Users/Rohil/Documents/iGEM/yemen/sagemaker_input/y_46_preselect.csv')

In [206]:
orchestrator46.runRollingCustomFeatureExtractor()

Running custom feature selection module ....
Creating baseline model to extract features
all features (2940, 7910)
Thresholds are 7910 30 [0.01246512 0.00986047 0.00893023 0.00818605 0.008      0.00651163
 0.00613953 0.00595349 0.0055814  0.0052093  0.00502326 0.00446512
 0.00427907 0.00409302 0.00390698 0.00372093 0.00353488 0.00334884
 0.00316279 0.00297674 0.0027907  0.00260465 0.0024186  0.00223256
 0.00204651 0.00186047 0.00167442 0.00148837 0.00130233 0.00111628]
CV - Window 3
Train [2017-06-19 2017-08-18] Val [2017-08-19 2017-09-08] Test [2017-09-09 2017-09-29]
Window Train/Val/Test shape (1281, 7911) (441, 7911) (441, 7911)
Window Train/Val/Test shape (1281, 2) (441, 2) (441, 2)
X_train_cv  (1722, 7910)
3 Thresh=0.01247, n=1, mae: 6.439, mae: 0.6034
CV - Window 4
Train [2017-06-19 2017-09-07] Val [2017-09-08 2017-09-28] Test [2017-09-29 2017-10-19]
Window Train/Val/Test shape (1701, 7911) (441, 7911) (441, 7911)
Window Train/Val/Test shape (1701, 2) (441, 2) (441, 2)
X_train_cv

Window Train/Val/Test shape (1281, 7911) (441, 7911) (441, 7911)
Window Train/Val/Test shape (1281, 2) (441, 2) (441, 2)
X_train_cv  (1722, 7910)
3 Thresh=0.00391, n=21, mae: 3.974, mae: 0.8300
CV - Window 4
Train [2017-06-19 2017-09-07] Val [2017-09-08 2017-09-28] Test [2017-09-29 2017-10-19]
Window Train/Val/Test shape (1701, 7911) (441, 7911) (441, 7911)
Window Train/Val/Test shape (1701, 2) (441, 2) (441, 2)
X_train_cv  (2142, 7910)
4 Thresh=0.00391, n=21, mae: 3.375, mae: 0.8365
CV - Window 5
Rolling window to end date
Train [2017-06-19 2017-09-27] Val [2017-09-28 2017-10-18] Test [2017-10-19 2017-11-10]
Window Train/Val/Test shape (2121, 7911) (441, 7911) (483, 7911)
Window Train/Val/Test shape (2121, 2) (441, 2) (483, 2)
X_train_cv  (2562, 7910)
5 Thresh=0.00391, n=21, mae: 1.774, mae: 0.9066
{'threshold': 0.003906977, 'num_features': 21, 'mae3': 3.973605666637657, 'mae4': 3.374618222384309, 'mae5': 1.77354025903554, 'ev3': 0.8299999156811699, 'ev4': 0.8365186597319431, 'ev5': 0

Train [2017-06-19 2017-09-27] Val [2017-09-28 2017-10-18] Test [2017-10-19 2017-11-10]
Window Train/Val/Test shape (2121, 7911) (441, 7911) (483, 7911)
Window Train/Val/Test shape (2121, 2) (441, 2) (483, 2)
X_train_cv  (2562, 7910)
5 Thresh=0.00130, n=118, mae: 2.034, mae: 0.8963
{'threshold': 0.0013023256, 'num_features': 118, 'mae3': 4.237570823599514, 'mae4': 3.0873436698245063, 'mae5': 2.034342081581877, 'ev3': 0.793830882854738, 'ev4': 0.8831669920988946, 'ev5': 0.896296468208437}


CV - Window 3
Train [2017-06-19 2017-08-18] Val [2017-08-19 2017-09-08] Test [2017-09-09 2017-09-29]
Window Train/Val/Test shape (1281, 7911) (441, 7911) (441, 7911)
Window Train/Val/Test shape (1281, 2) (441, 2) (441, 2)
X_train_cv  (1722, 7910)
3 Thresh=0.00112, n=154, mae: 4.364, mae: 0.7773
CV - Window 4
Train [2017-06-19 2017-09-07] Val [2017-09-08 2017-09-28] Test [2017-09-29 2017-10-19]
Window Train/Val/Test shape (1701, 7911) (441, 7911) (441, 7911)
Window Train/Val/Test shape (1701, 2) (441, 

In [208]:
orchestrator46.rcfe.summary[['mean_mae', 'mean_ev'] ]= pd.concat([orchestrator46.rcfe.summary[['mae3', 'mae4', 'mae5']].mean(axis=1), orchestrator46.rcfe.summary[['ev3', 'ev4', 'ev5']].mean(axis=1)], axis=1)

In [209]:
orchestrator46.rcfe.summary

threshold  num_features      mae3      mae4      mae5       ev3       ev4  \
0    0.012465           1.0  6.439048  6.726006  3.932054  0.603398  0.419194   
1    0.009860           2.0  7.633216  7.354925  3.196735  0.464808  0.301770   
2    0.008930           3.0  5.334601  5.693728  2.654387  0.727467  0.594808   
3    0.008186           4.0  4.166086  4.643442  2.576561  0.825348  0.702110   
4    0.008000           5.0  4.254721  4.247427  2.209554  0.782887  0.719452   
5    0.006512           6.0  4.412089  4.098490  2.122569  0.770113  0.737685   
6    0.006140           7.0  4.532120  4.056695  2.201728  0.767413  0.739081   
7    0.005953           9.0  3.962170  3.647813  2.140954  0.816339  0.807882   
8    0.005581          10.0  3.785105  3.550510  2.313031  0.850732  0.827515   
9    0.005209          12.0  4.033114  3.595842  1.931865  0.837829  0.824713   
10   0.005023          14.0  3.880816  3.512000  1.946589  0.833139  0.815087   
11   0.004465          16.0  3.756105  3.440991  2.090706  0.850803  0.818181   
12   0.004279          17.0  3.809822  3.375033  2.017410  0.842153  0.834681   
13   0.004093          20.0  3.882743  3.385603  1.836640  0.842080  0.841590   
14   0.003907          21.0  3.973606  3.374618  1.773540  0.830000  0.836519   
15   0.003721          24.0  3.769952  3.354888  1.782541  0.838415  0.839253   
16   0.003535          25.0  3.783856  3.359148  1.799377  0.834601  0.833987   
17   0.003349          29.0  3.827671  3.322616  1.750044  0.839699  0.843702   
18   0.003163          32.0  3.935942  3.437171  1.793171  0.840811  0.834204   
19   0.002977          35.0  3.708983  3.334238  1.749874  0.848440  0.840497   
20   0.002791          39.0  3.812976  3.274766  1.683258  0.836123  0.852579   
21   0.002605          41.0  3.817030  3.180274  1.734318  0.838075  0.870734   
22   0.002419          47.0  3.968745  3.261961  1.781060  0.823966  0.853611   
23   0.002233          54.0  4.010965  3.216443  1.719254  0.827553  0.856166   
24   0.002047          62.0  4.172439  3.276562  1.798191  0.807001  0.859486   
25   0.001860          70.0  3.978083  3.251501  1.759377  0.819199  0.857090   
26   0.001674          83.0  4.230017  3.158038  1.620880  0.799790  0.870860   
27   0.001488          96.0  4.034087  3.249504  1.764031  0.815712  0.858347   
28   0.001302         118.0  4.237571  3.087344  2.034342  0.793831  0.883167   
29   0.001116         154.0  4.364363  3.130318  2.106362  0.777345  0.863929   

         ev5  mean_mae   mean_ev  
0   0.602591  5.699036  0.541728  
1   0.704902  6.061626  0.490494  
2   0.758464  4.560906  0.693580  
3   0.776820  3.795363  0.768093  
4   0.834489  3.570567  0.778943  
5   0.824144  3.544382  0.777314  
6   0.823395  3.596848  0.776630  
7   0.853616  3.250312  0.825946  
8   0.833697  3.216215  0.837315  
9   0.903131  3.186940  0.855224  
10  0.888815  3.113135  0.845681  
11  0.858393  3.095934  0.842459  
12  0.869684  3.067422  0.848839  
13  0.893126  3.034995  0.858932  
14  0.906587  3.040588  0.857702  
15  0.902934  2.969127  0.860201  
16  0.903588  2.980793  0.857392  
17  0.903728  2.966777  0.862376  
18  0.904053  3.055428  0.859689  
19  0.905235  2.931032  0.864724  
20  0.919860  2.923667  0.869520  
21  0.912337  2.910541  0.873715  
22  0.909593  3.003922  0.862390  
23  0.920659  2.982221  0.868126  
24  0.916221  3.082397  0.860903  
25  0.919059  2.996320  0.865116  
26  0.928342  3.002978  0.866330  
27  0.909766  3.015874  0.861275  
28  0.896296  3.119752  0.857765  
29  0.896754  3.200348  0.846009

In [236]:
# orchestrator462 = Orchestrator(full_features, full_data_4_6, 'week_4_to_6_cases' )
# orchestrator462.tf = orchestrator46.tf
# orchestrator462.op = orchestrator46.op
# orchestrator462.corr_selected_features_bi_week = orchestrator46.corr_selected_features_bi_week
# orchestrator462.corr_selected_features_bi_week_y = orchestrator46.corr_selected_features_bi_week_y
# orchestrator462.rcfe = orchestrator46.rcfe

In [298]:
selected_features46 = ['date'] + list(orchestrator46.rcfe.mcr.feature_importance_df[orchestrator46.rcfe.mcr.feature_importance_df.threshold >= 0.002605].index)

In [299]:
orchestrator462.corr_selected_features_bi_week[selected_features46].to_csv('/Users/Rohil/Documents/iGEM/yemen/sagemaker_input/X_4_6.csv')

In [300]:
orchestrator462.corr_selected_features_bi_week_y.to_csv('/Users/Rohil/Documents/iGEM/yemen/sagemaker_input/y_4_6.csv')

In [271]:
orchestrator462.deploy(0.002605)

C:\Users\Rohil\Anaconda3\lib\site-packages\ipykernel_launcher.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Creating baseline model to train on final features
y-test mean 5.381826168324619, y-test std 6.315341352329288
xgb mae 4.234231543912594, xgb ev 0.44332574984962136
linreg mae 6.156414259972813, linreg ev -0.6065695828730608


In [49]:
# running feature selection for 6-8 week model

In [50]:
orchestrator68 = Orchestrator(full_features, full_data_6_8, 'week_6_to_8_cases' )

In [145]:
# orchestrator68 = Orchestrator(full_features, full_data_6_8, 'week_6_to_8_cases' )
# orchestrator68.tf = orchestrator682.tf
# orchestrator68.op = orchestrator682.op
# orchestrator68.corr_selected_features_bi_week = orchestrator682.corr_selected_features_bi_week
# orchestrator68.corr_selected_features_bi_week_y = orchestrator682.corr_selected_features_bi_week_y
# orchestrator68.rcfe = orchestrator682.rcfe

In [51]:
orchestrator68.runTsFresh()

Running TSFresh....
Selected features (3717, 15087)
Finished running TSFresh....


In [52]:
orchestrator68.runOptimizeFeatures()

Running Feature Selection module ....
4719 features with a correlation magnitude greater than 0.97.

Original (3717, 15088) and after (3717, 10369)
Finished running Feature Selection ....


In [68]:
X_68_preselect = orchestrator68.op.corr_selected_features
y_68_preselect = orchestrator68.full_data_bi_week[['date', orchestrator68.y_bi_week_label]]

In [69]:
X_68_preselect.to_csv('/Users/Rohil/Documents/iGEM/yemen/sagemaker_input/X_68_preselect.csv')
y_68_preselect.to_csv('/Users/Rohil/Documents/iGEM/yemen/sagemaker_input/y_68_preselect.csv')

In [85]:
orchestrator68.rcfe.summary[['mean_mae', 'mean_ev'] ]= pd.concat([orchestrator68.rcfe.summary[['mae3', 'mae4', 'mae5']].mean(axis=1), orchestrator68.rcfe.summary[['ev3', 'ev4', 'ev5']].mean(axis=1)], axis=1)

In [124]:
orchestrator68.rcfe.summary

threshold  num_features      mae3      mae4      mae5       ev3       ev4  \
0    0.011879           1.0  4.268170  4.406226  4.116368  0.824195  0.545805   
1    0.007463           2.0  6.539971  4.710951  3.449654  0.514628  0.455287   
2    0.007158           4.0  5.010040  3.432586  2.230406  0.719831  0.711694   
3    0.006701           5.0  5.297530  3.623473  2.196311  0.674772  0.697964   
4    0.006244           6.0  4.388213  3.101935  2.382979  0.784948  0.711002   
5    0.005787           7.0  4.455287  3.343316  2.455334  0.786149  0.685313   
6    0.005483           8.0  4.298365  3.184772  2.372711  0.790018  0.692693   
7    0.005026           9.0  4.202619  3.061092  2.419323  0.808198  0.721014   
8    0.004569          10.0  4.057791  2.911899  2.511658  0.814124  0.737887   
9    0.004417          11.0  4.163161  3.062369  2.166033  0.818813  0.713195   
10   0.004264          12.0  4.159501  3.041249  2.311916  0.805297  0.727593   
11   0.004112          14.0  4.301015  2.821911  2.318344  0.804137  0.777014   
12   0.003960          18.0  4.315318  2.723214  2.371798  0.787122  0.784723   
13   0.003807          19.0  4.378722  3.039504  2.208939  0.781227  0.743553   
14   0.003655          20.0  3.944676  3.162269  2.319018  0.835563  0.690625   
15   0.003198          21.0  4.173465  3.110525  2.251499  0.802313  0.692705   
16   0.003046          22.0  4.077949  3.086000  2.293193  0.818897  0.715565   
17   0.002894          25.0  4.013783  3.007644  2.276844  0.814995  0.727100   
18   0.002741          27.0  4.126205  2.987734  2.226597  0.798593  0.725542   
19   0.002589          31.0  4.082440  3.061749  2.282711  0.789316  0.703717   
20   0.002437          33.0  4.023751  2.923378  2.390902  0.802143  0.742157   
21   0.002284          40.0  3.786937  2.831377  2.106199  0.811881  0.768220   
22   0.002132          48.0  3.963609  2.856466  2.178423  0.792990  0.765069   
23   0.001980          55.0  4.230286  2.931346  2.150468  0.756836  0.757656   
24   0.001828          66.0  4.339570  2.731545  2.209070  0.743068  0.783480   
25   0.001675          80.0  4.378284  2.805200  2.301862  0.735501  0.768331   
26   0.001523          96.0  4.220433  2.775102  2.251989  0.758241  0.781269   
27   0.001371         115.0  4.271255  2.811183  2.320075  0.751195  0.770072   
28   0.001218         139.0  4.070151  2.781837  2.262053  0.781005  0.782973   
29   0.001066         177.0  4.219280  2.728146  2.239048  0.771681  0.793878   

         ev5  mean_mae   mean_ev  
0   0.420275  4.263588  0.596759  
1   0.496529  4.900192  0.488815  
2   0.844706  3.557677  0.758744  
3   0.842191  3.705771  0.738309  
4   0.826010  3.291042  0.773987  
5   0.800995  3.417979  0.757486  
6   0.809001  3.285282  0.763904  
7   0.798619  3.227678  0.775944  
8   0.777737  3.160449  0.776583  
9   0.836734  3.130521  0.789580  
10  0.811585  3.170889  0.781492  
11  0.818667  3.147090  0.799939  
12  0.810787  3.136777  0.794211  
13  0.822856  3.209055  0.782545  
14  0.816388  3.141988  0.780859  
15  0.816627  3.178497  0.770549  
16  0.810557  3.152381  0.781673  
17  0.815957  3.099424  0.786017  
18  0.823415  3.113512  0.782516  
19  0.818003  3.142300  0.770345  
20  0.813030  3.112677  0.785777  
21  0.866500  2.908171  0.815534  
22  0.850882  2.999500  0.802980  
23  0.855365  3.104033  0.789952  
24  0.822496  3.093395  0.783014  
25  0.801555  3.161782  0.768463  
26  0.809287  3.082508  0.782932  
27  0.808744  3.134171  0.776670  
28  0.813174  3.038014  0.792384  
29  0.823012  3.062158  0.796190

In [290]:
selected_features68 = ['date'] + list(orchestrator68.rcfe.mcr.feature_importance_df[orchestrator68.rcfe.mcr.feature_importance_df.threshold >= 0.002284].index)

In [291]:
orchestrator68.corr_selected_features_bi_week[selected_features68].to_csv('/Users/Rohil/Documents/iGEM/yemen/sagemaker_input/X_6_8.csv')

In [292]:
orchestrator68.corr_selected_features_bi_week_y.to_csv('/Users/Rohil/Documents/iGEM/yemen/sagemaker_input/y_6_8.csv')

In [293]:
orchestrator68.deploy(0.002284)

Creating baseline model to train on final features
y-test mean 4.671585825506454, y-test std 5.5720761674820904
xgb mae 3.1162140401978666, xgb ev 0.5350739582444819
linreg mae 4.027808065496075, linreg ev 0.1196188209004242


In [104]:
orchestrator68.corr_selected_features_bi_week_y.sort_values(by='date')

date  week_6_to_8_cases
2456 2017-06-19           7.143574
2441 2017-06-19          49.379311
2444 2017-06-19          17.993787
2443 2017-06-19          44.854502
2442 2017-06-19           3.873987
2446 2017-06-19          22.996410
2439 2017-06-19          28.036947
2438 2017-06-19          70.382174
2437 2017-06-19          10.251431
2445 2017-06-19          10.268290
2440 2017-06-19          75.670455
2447 2017-06-19          38.913257
2436 2017-06-19          50.711571
2454 2017-06-19           1.857227
2453 2017-06-19           8.155616
2452 2017-06-19          57.602912
2455 2017-06-19           7.362924
2450 2017-06-19          71.331039
2449 2017-06-19           5.562500
2448 2017-06-19           9.649373
2451 2017-06-19          22.178884
2468 2017-06-20          36.457226
2463 2017-06-20           3.292889
2466 2017-06-20          10.058628
2465 2017-06-20          17.377879
2464 2017-06-20          43.100593
2459 2017-06-20          69.986143
2462 2017-06-20          49.550195
2461 2017-06-20          75.255595
2460 2017-06-20          26.792456
...         ...                ...
2411 2017-12-23           0.000000
2406 2017-12-23           0.033967
2413 2017-12-23           3.128906
2412 2017-12-23           0.000000
2414 2017-12-23           0.547033
2409 2017-12-23           7.960120
2408 2017-12-23           8.487239
2407 2017-12-23           0.000000
2410 2017-12-23          10.668377
2422 2017-12-24           0.015896
2421 2017-12-24           0.038740
2419 2017-12-24           0.016594
2415 2017-12-24           0.030357
2417 2017-12-24           3.639920
2416 2017-12-24           0.312600
2423 2017-12-24           4.360364
2418 2017-12-24           7.036161
2424 2017-12-24           2.510822
2430 2017-12-24           7.880519
2426 2017-12-24           0.000000
2428 2017-12-24           0.000000
2429 2017-12-24           8.356546
2431 2017-12-24          10.844704
2432 2017-12-24           0.000000
2433 2017-12-24           0.000000
2434 2017-12-24           3.153972
2427 2017-12-24           0.039628
2425 2017-12-24           0.919856
2420 2017-12-24           6.063402
2435 2017-12-24           0.524762

[3969 rows x 2 columns]

In [ ]:
import sagemaker                                  # Amazon SageMaker's Python SDK provides many helper functions
from sagemaker.predictor import csv_serializer    # Converts strings for HTTP POST requests on inference

from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

prefix = 'sagemaker/xgboost-ts'

In [ ]:
deployRegressor = True
if deployRegressor == True:
    X = corr_selected_features_4_6[selected_features]
    y = corr_selected_features_4_6_y 
else:
    X = corr_selected_features_4_6 
    y = corr_selected_features_4_6_y 
    
X_train, X_cv, X_test = train_cv_test_split(X)
y_train, y_cv, y_test = train_cv_test_split(y)

In [35]:
X_train.head()

In [ ]:
y_train.head()

In [ ]:
pd.concat([y_train['week_4_to_6_cases'], X_train], axis=1).to_csv('train.csv', index=False, header=False)
pd.concat([y_cv['week_4_to_6_cases'], X_cv], axis=1).to_csv('validation.csv', index=False, header=False)
pd.concat([y_test['week_4_to_6_cases'], X_test], axis=1).to_csv('test.csv', index=False, header=False)


In [ ]:
# Upload for Sagemaker jobs to pickup
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'validation/validation.csv')).upload_file('validation.csv')

In [ ]:
s3_input_train = sagemaker.s3_input(s3_data='s3://{}/{}/train'.format(bucket, prefix), content_type='csv')
s3_input_validation = sagemaker.s3_input(s3_data='s3://{}/{}/validation/'.format(bucket, prefix), content_type='csv')
print('s3://{}/{}/train'.format(bucket, prefix))

In [ ]:
from sagemaker.amazon.amazon_estimator import get_image_uri

sess = sagemaker.Session()

container = get_image_uri(region, 'xgboost')

xgb = sagemaker.estimator.Estimator(container,
                                    role, 
                                    train_instance_count=1, 
                                    train_instance_type='ml.m4.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    sagemaker_session=sess)

In [ ]:
# Static parameters
# 'auc'
# eval_metric: Metric 'multi:softmax' is not supported. 
# Parameter 'eval_metric' should be one of these options:'rmse', 'mae', 'logloss', 'error', 'merror', 'mlogloss', 'auc', 'ndcg', 'map', 'ndcg-', 'map-', 'poisson-nloglik', 'gamma-nloglik', 'gamma-deviance', 'tweedie-nloglik'.
xgb.set_hyperparameters(eval_metric='mae', 
                        max_depth=3,
                        eta=0.1,
                        gamma=4,
                        min_child_weight=5,
                        subsample=0.9,
                        colsample_bytree=0.8,
                        silent=0,
                        objective='reg:linear',
                        num_round=500,
                        early_stopping_rounds=100,
                        rate_drop=0.3,
                        tweedie_variance_power=1.4)

#max_depth=[3,5,7]
#subsample=[0.7,0.8,0.9],
#colsample_bytree=[0.7,0.8,0.9],
hyperparameter_ranges = {
    'eta': ContinuousParameter(0.0, 1.0),
    'min_child_weight': ContinuousParameter(1.0, 10.0),
    'alpha': ContinuousParameter(0, 2),
    'max_depth': IntegerParameter(3, 12),
    'subsample': ContinuousParameter(0.3, 0.9),
    'colsample_bytree': ContinuousParameter(0.3, 0.9),
    'num_round': IntegerParameter(50, 1000)
}

In [ ]:
#https://sagemaker.readthedocs.io/en/latest/tuner.html
tuner = HyperparameterTuner(xgb,
                            objective_metric_name = 'validation:mae',
                            objective_type = 'Minimize',
                            strategy='Bayesian',
                            hyperparameter_ranges = hyperparameter_ranges,
                            max_jobs=20,
                            max_parallel_jobs=5)

In [ ]:
# Fit with train & validation data sets
tuner.fit({'train': s3_input_train, 'validation': s3_input_validation})

In [ ]:
#Let's just run a quick check of the hyperparameter tuning jobs status to make sure it started successfully.
boto3.client('sagemaker').describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=tuner.latest_tuning_job.job_name)['HyperParameterTuningJobStatus']

In [ ]:
#Deploy the best trained model
xgb_predictor = tuner.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

In [ ]:
xgb_predictor.content_type = 'text/csv'
xgb_predictor.serializer = csv_serializer

In [ ]:
def predict(data, rows=500):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ''
    for array in split_array:
        predictions = ','.join([predictions, xgb_predictor.predict(array).decode('utf-8')])

    return np.fromstring(predictions[1:], sep=',')

predictions = predict(X_test.as_matrix()[:, 1:])
len(predictions)

In [ ]:
print('Test sample X {} y {} Predictions {}'.format(X_test.shape, y_test.shape, len(predictions)))

In [ ]:
mean_absolute_error(y_test['week_4_to_6_cases'], predictions)

In [ ]:
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)

In [ ]:
a = [1,5,2,7,5,4, 4]
a = np.unique(a)
a.sort()
a=a[::-1]
a

In [ ]:

df = pd.DataFrame(columns = ['threshold', 'nf', 'mae3', 'mae4', 'mae5' ], index=None)

a = [2.3, 3]
b = [1.5, 2]
c = [2]
series = pd.Series(a + b+ c)
df = df.append(series, ignore_index = True)

In [ ]:
df

In [ ]:
dict(zip(['threshold', 'num_features', 'mae3', 'mae4', 'mae5', 'ev3', 'ev4', 'ev5'], a + b+ c))